# Esercitazione "Laboratorio di Programmazione Geometrica e Grafica"
## Workshop_09 (18-12-17)

# Villa Pisani di Bagnolo

Link Wikipedia:  <a href="https://it.wikipedia.org/wiki/Villa_Pisani_(Bagnolo)">Villa Pisani (Bagnolo)</a> 

- In questo workshop mi sono occupato di ricostruire Villa Pisani di Bagnolo utilizzando gran parte degli stumenti appresi durante il corso;
- Per svolgere l'esercizio ho definito diverse funzioni parametriche con lo scopo di poterle utilizzare per ottenere il risultato finale;
- Per fare questo ho definito funzioni per creare, tramite passaggio di parametri: finestre, porte, decorazioni, pareti, scalinate, pavimenti, tetti, alberi ecc.
- E' possibile variare diversi parametri per la generazione della Villa senza compromettere il risultato finale, come spessore dei muri, altezza dei piani, le dimensioni dell'edificio, la dimensione del prato, il numero di alberi ecc.
- Ogni componente può essere riutilizzata per costruire edifici simili, ad esempio è possibile costruire un muro lungo 30 metri, con un determinato tipo di finestre, semplicemente chiamando l'opportuna funzione. Allo stesso modo è possibile creare una serie di archi (come quelli utilizzati nella facciata dell'edificio), una scalinata con un numero arbitrario di gradini e di altezza, profondità e larghezza variabile e molti altri elementi architettonici. Tutti gli elementi che compaiono del risultato finale sono riutilizzabili;
- Inoltra sono state definite le pareti interne all'edificio per completare il lavoro della ricostruzione della Villa, con tanto di varchi e scale per collegare i diversi piani, il tutto seguendo la planimetria;
- Sono state utilizzate delle texture per rendere più verosimile il risultato finale. Anche le texture possono essere semplicemente modificate della sezione apposita;
- In fine ho creato una funzione che dato uno spazio di prato libero (anche questo ricavato in modo parametrico tramite l'utilizzo di un'opportuna funzione) genera un determinato numero di alberi di diverso colore e dimensione in posizioni casuali della mappa. In questo modo è stato possibile utilizzare Numpy e le funzioni random.

<table>
    <tr><td>![villa_pisani_frontale.jpg](imm/originale/villa_pisani_frontale.jpg)</td></tr>
    <tr><td>![villa_pisani_retro.jpg](imm/originale/villa_pisani_retro.jpg)</td></tr>
    <tr><td>![villa_pisani_laterale.jpg](imm/originale/villa_pisani_laterale.jpg)</td></tr>
    <tr><td>![villa_pisani_2.jpg](imm/originale/villa_pisani_2.jpg)</td></tr>
</table>

### Import di Larlib e Numpy

In [1]:
from larlib import *
from numpy import *

Evaluating fenvs.py..
...fenvs.py imported in 0.00676305371292 seconds


c:\programmi\python27\lib\site-packages\larlib\larstruct.py:233: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  self.body = [item for item in data if item != None]


### Misure di riferimento

In [2]:
# Misure di riferimento

spessore_muri = 0.3

dimensione_pannello_finestra = 8

altezza_primo_piano = 14

spessore_facciata = 2

altezza_tetti = 1.5

altezza_scalinata_frontale = 2
raggio_scalinata_frontale = 5.9

dim_prato_x = 80
dim_prato_y = 100

### Misure ricavate
- queste misure vengono calcolate dopo la definizione delle misure di riferimento così da essere richiamate successivamente snellendo il codice.

In [3]:
# Misure ricavate

numero_finestre_laterali = 4
numero_finestre_posteriori = 6

dimensione_lato_frontale_edificio = dimensione_pannello_finestra*1.0*numero_finestre_posteriori
dimensione_laterale_edificio = dimensione_pannello_finestra*1.0*numero_finestre_laterali

DimSpazioBianco = ((dimensione_pannello_finestra-2)/2.0)-2

posizione_piano2_X = (dim_prato_x-(dimensione_laterale_edificio+2*DimSpazioBianco))/2.0
posizione_piano2_Y = (dim_prato_y-(dimensione_lato_frontale_edificio+2*DimSpazioBianco))/2.0

posizione_facciata_Y = (((dim_prato_x-(dimensione_laterale_edificio+2.0*DimSpazioBianco))/2.0)+dimensione_laterale_edificio+DimSpazioBianco*2.0)-spessore_facciata
posizione_facciata_Z = 3+0.3+spessore_muri*3
posizione_facciata_X = posizione_piano2_Y + dimensione_pannello_finestra + DimSpazioBianco*2.0

altezza_piano_0 = 3+0.5+spessore_muri*3.0

altezza_secondo_piano = 3+0.3+spessore_muri*4+altezza_primo_piano+0.1

altezza_tetto = altezza_secondo_piano + altezza_primo_piano*0.4+spessore_muri
posizione_tetto_X = posizione_piano2_X+dimensione_laterale_edificio-dimensione_pannello_finestra
posizione_tetto_Y = posizione_piano2_Y

altezza_pareti = altezza_secondo_piano + altezza_primo_piano*0.6

### Texture e colori
- le texture possono essere modificate in questo punto del codice.

In [4]:
# Texture e colori

texture_pietra = 'texture/sand3.png'
texture_pietraP0 = 'texture/mattoni3.png'
texture_prato = 'texture/prato.png'
texture_brecciolino = 'texture/brecciolino2.png'
texture_tetto = 'texture/tetto3.png'
texture_pavimentoP1 = 'texture/marmo.png'
texture_pavimentoP2 = 'texture/marmo.png'
texture_pareti = 'texture/marmo-bianco.png'
texture_finestre = 'texture/marmo-bianco.png'
texture_persiane = 'texture/legno.png'

texture_foglie1 = 'texture/foglie1.png'
texture_foglie2 = 'texture/foglie2.png'
texture_foglie3 = 'texture/foglie3.png'
texture_tronco = 'texture/tronco.png'

### Funzioni di supporto
- alcune funzioni che successivamente verranno richiamate per la definizione di funzioni più complesse.

In [5]:
# Mappa ciclica

def cylMap():
    x = lambda k: k[1] * COS(k[0])
    y = lambda k: k[1] * SIN(k[0])
    z = lambda k: k[2]
    return MAP([x,y,z])

In [6]:
# Alcuni metodi di supporto

def getMapSemiarco(r,altezza):
    x = lambda p : r*sin(p[0])*cos(p[0])
    y = lambda p : r*cos(p[0])*cos(p[0])
    z = lambda p : altezza
    dom = INTERVALS(PI/2)(63) # primo parametro dimensione dominio, secondo parametro in quante parti voglio dividerlo

    map = MAP([x,y,z])(dom)
    return map

def getMap(r,altezza):
    x = lambda p : r*sin(p[0])*cos(p[0])
    y = lambda p : r*cos(p[0])*cos(p[0])
    z = lambda p : altezza
    dom = INTERVALS(PI)(32) # primo parametro dimensione dominio, secondo parametro in quante parti voglio dividerlo

    map = MAP([x,y,z])(dom)
    
    return map

def Cubo_Angoli_Smussati(r,altezza,fattore_smussamento):
    fattore_smussamento = fattore_smussamento * r
    elemento = OFFSET([fattore_smussamento,fattore_smussamento,fattore_smussamento])(JOIN([getMap(r,0),getMap(r,altezza)]))
    
    return elemento

def getPrato(x,y):
    prato = TEXTURE(texture_prato)(CUBOID([x,y,0.1]))
    brecciolino = TEXTURE(texture_brecciolino)(T(2)(dim_prato_y/2.0-((dimensione_pannello_finestra*2.0-DimSpazioBianco*2.0)/2.0))(CUBOID([posizione_piano2_Y,dimensione_pannello_finestra*2.0-DimSpazioBianco*2.0,0.15])))
    base = STRUCT([prato, brecciolino])
    return base

Prato = getPrato(dim_prato_x,dim_prato_y)

def getCollinetta(altezza,lunghezza_totale, lunghezza_cima,spessore):
    lato = (lunghezza_totale-lunghezza_cima)*1.0/2.0
    
    V1 = [[0,0],[lato,altezza],[lato+lunghezza_cima,altezza],[lunghezza_totale,0]]
    FV = [[0,1],[1,2],[2,3],[3,0]]
   
    Collina = JOIN(STRUCT(MKPOLS((V1,FV))))
    Collina = T(2)(-(lunghezza_totale/2.0))(R([1,2])(PI/2)(R([2,3])(PI/2)(MULTEXTRUDE(Collina)(spessore))))
    
    return TEXTURE(texture_prato)(Collina)

Collinetta = T([1,2])([posizione_facciata_Y+spessore_facciata,dim_prato_y/2.0])(getCollinetta(2.5,40,14,posizione_piano2_X))

PratoConCollina = TEXTURE(texture_prato)(STRUCT([Prato,Collinetta]))

### Definizione del vetro
- restituisce un CUBOID trasparente di dimensioni x, y, z;
- utilizzato per i vetri delle finestre.

In [7]:
# Vetro

def hex_material(color, light, trasparence):

    def hex_to_rgb(value):
        value = value.lstrip('#')
        lv = len(value)
        return map(lambda x: x/255., list(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3)))

    params = hex_to_rgb(color) + [.1] + \
             hex_to_rgb(light) + [trasparence] +\
             hex_to_rgb(light) + [.1] +\
             hex_to_rgb("#000000") + [.1] +\
             [100]

    return MATERIAL(params)

def getVetro(altezza,larghezza,spessore):
    vetro = CUBOID([altezza,spessore,larghezza])
    vetro = hex_material("#ffffff", "#ffffff", .4)(vetro)
    return vetro

In [8]:
VIEW(getVetro(1,0.05,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D14C990> >

<table>
    <tr><td>![vetro.png](imm/vetro.png))</td>
    <td>![vetro.png](imm/vetro-2.png)</td></tr>
</table>

### Finestrella del piano 0
- getFinestrella(x,y) restituisce un muro composto da y parti di dimensione x;
- Ogni parte ha una propria finestra;
- per la creazione della finesrtra è stato utilizzato il prodotto cartesiano per crare una parte di muro forata;
- getVetro restituisce il vetro della finestra;
- sono poi state definite altre decorazioni come le sbarre ecc.
- getMattoniLaterali() restituisce una composizione di mattoni come quella della Villa.

In [9]:
# Finistrella del piano 0

def getMattoniLaterali():
    
    MattoneFinestraPiccolo = CUBOID([0.5,0.2,0.3])
    MattoneFinestraGrande = CUBOID([0.8,0.2,0.3])
    getMattoniLaterali = STRUCT([MattoneFinestraPiccolo,
                              T(3)(-0.35)(MattoneFinestraGrande),
                              T(3)(-0.35*2)(MattoneFinestraPiccolo),
                              T(3)(-0.35*3)(MattoneFinestraGrande),
                              T(3)(-0.35*4)(MattoneFinestraPiccolo)
                             ])
    return TEXTURE(texture_pietraP0)(getMattoniLaterali)

def getMattoniLaterali2():
    
    MattoneFinestraPiccolo = CUBOID([0.5,0.2,0.3])
    MattoneFinestraGrande = CUBOID([0.8,0.2,0.3])
    getMattoniLaterali = STRUCT([T(3)(0.35)(MattoneFinestraGrande),
                              MattoneFinestraPiccolo,
                              T(3)(-0.35)(MattoneFinestraGrande),
                              T(3)(-0.35*2)(MattoneFinestraPiccolo),
                              T(3)(-0.35*3)(MattoneFinestraGrande),
                              T(3)(-0.35*4)(MattoneFinestraPiccolo),
                              T(3)(-0.35*5)(MattoneFinestraGrande)
                             ])
    return getMattoniLaterali

def getFinestrella(lunghezza_pannello_finestra_p0, numero_pannelli):

    Finestrella1 = STRUCT([
        INSR(PROD)([QUOTE([0,2,-2,(lunghezza_pannello_finestra_p0-2)/2.0]), QUOTE([0,spessore_muri]), QUOTE([3])]),
        INSR(PROD)([QUOTE([-2,2]), QUOTE([0,spessore_muri]), QUOTE([0.6,-1.8,0.6])])
        ])
    
    DimensioneSpazioBianco = ((lunghezza_pannello_finestra_p0-2)/2.0)-2
    SpazioBianco = CUBOID([DimensioneSpazioBianco,spessore_muri,3])
    
    Finestrella = LEFT([Finestrella1,SpazioBianco])
    
    RinghieraFinestrella = T([1,2,3])([2,spessore_muri/2,0.5])(COMP([STRUCT, NN(10)])([COLOR(GRAY)(Cubo_Angoli_Smussati(0.04,2,1)), T(1)(0.2)]))

    MattoniLaterali = getMattoniLaterali()

    MattoneFinestraPiccolo = CUBOID([0.5,0.2,0.3])
    MattoneFinestraGrande = CUBOID([0.8,0.2,0.3])
   
    MattoniFinestra = TEXTURE(texture_pietraP0)((T([1,2,3])([4,spessore_muri,2])(STRUCT([T([1,3])([0.05,0.05])(MattoniLaterali),
                                                              T([1,3])([-2.05,-1.05])(R([1,3])(PI)(MattoniLaterali)),
                                                              T([1,3])([-0.5,-0.35*5])(MattoneFinestraGrande),
                                                              T([1,3])([-1.35,-0.35*5])(MattoneFinestraGrande),
                                                              T([1,3])([-2.25,-0.35*5])(MattoneFinestraGrande),
                                                              T([1,3])([-0.5,0.45])(MattoneFinestraGrande),
                                                              T([1,3])([-1.35,0.45])(MattoneFinestraGrande),
                                                              T([1,3])([-2.25,0.45])(MattoneFinestraGrande)
                                                             ]))))
    
    FinestrellaCompleta = T(1)(((lunghezza_pannello_finestra_p0-2)/2.0)-2)(STRUCT([Finestrella, RinghieraFinestrella, MattoniFinestra]))
    
    DimSpazioBianco = ((lunghezza_pannello_finestra_p0-2)/2.0)-2

    SpazioBiancop0 = CUBOID([DimSpazioBianco,spessore_muri,3])

    ElementoMuro = FinestrellaCompleta

    Muro = RIGHT([SpazioBiancop0,RIGHT([COMP([STRUCT, NN(numero_pannelli)])([ElementoMuro, T(1)(lunghezza_pannello_finestra_p0)]),SpazioBiancop0])])
    
    return Muro

In [10]:
VIEW(getFinestrella(6,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1547B0> >

<table>
    <tr><td>![finestrella.png](imm/finestrella.png)</td>
    <td>![finestrella.png](imm/finestrella-2.png)</td></tr>
   
</table>

### Finestra del primo piano
- concettualmente simile alla definizione della finestra precedente;
- per creare la cornice decorativa intorno alle finestre è stato utilizzato MKPOLS.

In [11]:
# Finistra del piano 1

def getFinestraPrimoPiano (lunghezza_pannello_finestra_p1, altezza_pannello_finestra_p1, numero_pannelli_p1):

    Finestra1 = STRUCT([
        INSR(PROD)([QUOTE([0,2,-2,(lunghezza_pannello_finestra_p1-2)/2.0]), QUOTE([0,spessore_muri]), QUOTE([6])]),
        INSR(PROD)([QUOTE([-2,2]), QUOTE([0,spessore_muri]), QUOTE([0.9,-4.5,0.6])])
        ])

    DimSpazioBianco4 = ((lunghezza_pannello_finestra_p1-2)/2.0)-2
    SpazioBianco4 = CUBOID([DimSpazioBianco4,spessore_muri,altezza_pannello_finestra_p1])
 
    SpazioBianco2Sopra = CUBOID([lunghezza_pannello_finestra_p1,spessore_muri,((altezza_pannello_finestra_p1-6)*(2.0/3.0))])
    SpazioBianco2Sotto = CUBOID([lunghezza_pannello_finestra_p1,spessore_muri,((altezza_pannello_finestra_p1-6)*(1.0/3.0))])

    Finestra = LEFT([Finestra1,CUBOID([DimSpazioBianco4,spessore_muri,6])])

    Vfinestra = [[-0.15,0],[-0.15,6-0.6*2],[2.15,6-0.6*2],[2.15,0]]
    FVfinestra = [[0,1],[1,2],[2,3],[3,0]]

    ContornoFinestra1 = STRUCT(MKPOLS((Vfinestra,FVfinestra)))
    ContornoFinestraA = T([1,3,2])([2-0.05,0.6,0.5])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.1,0.1])(ContornoFinestra1))(0.1)))
    ContornoFinestraB = T([1,3,2])([2-0.1,0.6,0.4])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.2,0.2])(ContornoFinestra1))(0.2)))
    ContornoFinestraC = T([1,3,2])([2-0.15,0.6,0.3])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.3,0.3])(ContornoFinestra1))(0.3)))

    Mensola = TEXTURE(texture_finestre)(STRUCT([TOP([CUBOID([3,0.8,0.1]),CUBOID([3,0.5,0.1])])]))

    ContornoFinestra = TEXTURE(texture_finestre)(T(2)(spessore_muri)(STRUCT([ContornoFinestraA,ContornoFinestraB,ContornoFinestraC])))

    FinestraCompleta1 = T(1)(((lunghezza_pannello_finestra_p1-2)/2.0)-2)(STRUCT([Finestra,ContornoFinestra]))

    Vetro1 = getVetro(2,4.8,0.05)
    
    larghezza_persiane = 0.15
    Vlegno = [[0,0.3],[0,6-0.6*2-larghezza_persiane],[2-larghezza_persiane,6-0.6*2-larghezza_persiane],[2-larghezza_persiane,0.3],[(2-larghezza_persiane)/2.0,0.3], [(2-larghezza_persiane)/2.0,6-0.6*2-larghezza_persiane],[0,1.3],[2-larghezza_persiane,1.3],[0,3.5],[2-larghezza_persiane,3.5]]
    FVlegno = [[0,1],[1,2],[2,3],[3,0],[4,5],[6,7],[8,9]]

    Legno1 = STRUCT(MKPOLS((Vlegno,FVlegno)))
    Legno = TEXTURE(texture_persiane)(T([1,2,3])([larghezza_persiane,0.1,larghezza_persiane])((R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([-larghezza_persiane,-larghezza_persiane])(Legno1))(0.2)))))
    Vetro = STRUCT([Vetro1,Legno])
    
    #VIEW(Vetro)

    FinestraCompleta2 = STRUCT([FinestraCompleta1, T([1,2,3])([(lunghezza_pannello_finestra_p1-2)/2.0,spessore_muri,0.6])(Vetro), T([1,2,3])([((lunghezza_pannello_finestra_p1-2)/2.0)-0.5,spessore_muri-0.2,0.6-0.2])(Mensola), T([1,2,3])([((lunghezza_pannello_finestra_p1-2)/2.0)+2.5,spessore_muri-0.2,0.6*2+4.8])(R([1,3])(PI)(Mensola))])

    FinestraCompleta = T(3)(((altezza_pannello_finestra_p1-6)*(1/3.0)))(STRUCT([FinestraCompleta2, T(3)(6)(SpazioBianco2Sopra),T(3)(-((altezza_pannello_finestra_p1-6)*(1/3.0)))(SpazioBianco2Sotto)]))
    
    ElementoMuroPiano1 = FinestraCompleta
    
    MuroPrimoPiano = RIGHT([SpazioBianco4,RIGHT([COMP([STRUCT, NN(numero_pannelli_p1)])([ElementoMuroPiano1, T(1)(lunghezza_pannello_finestra_p1)]),SpazioBianco4])])

    return MuroPrimoPiano


In [12]:
VIEW(getFinestraPrimoPiano(6,8,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D154C00> >

<table>
    <tr><td>![finestraP1.png](imm/finestraP11.png)</td>
    <td>![finestraP1.png](imm/finestraP1-2.png)</td></tr>
</table>

### Porta del primo piano (retro)
- concettualmente simile alla definizione della finestrella del piano 0.

In [13]:
# Finestra e porta retro

def getPorta (lunghezza_pannello_porta_p1, altezza_pannello_porta_p1):
   
    Porta1 = STRUCT([
        INSR(PROD)([QUOTE([0,2,-2,(lunghezza_pannello_porta_p1-2)/2.0]), QUOTE([0,spessore_muri]), QUOTE([altezza_pannello_porta_p1])]),
        INSR(PROD)([QUOTE([-2,2]), QUOTE([0,spessore_muri]), QUOTE([-4.8,altezza_pannello_porta_p1-4.8])])
        ])
    
    DimSpazioBianco = ((lunghezza_pannello_porta_p1-2)/2.0)-2
    SpazioBianco = CUBOID([DimSpazioBianco,spessore_muri,altezza_pannello_porta_p1])
    
    Porta = LEFT([Porta1,SpazioBianco])

    Vporta = [[0,0],[0,6-0.6*2],[2,6-0.6*2],[2,0]]
    FVporta = [[0,1],[1,2],[2,3]]

    ContornoPorta1 = STRUCT(MKPOLS((Vporta,FVporta)))
    ContornoPortaA = T([1,2])([2-0.05,0.5])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.1,0.1])(ContornoPorta1))(0.1)))
    ContornoPortaB = T([1,2])([2-0.1,0.4])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.2,0.2])(ContornoPorta1))(0.2)))
    ContornoPortaC = T([1,2])([2-0.15,0.3])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.3,0.3])(ContornoPorta1))(0.3)))

    Mensola = TEXTURE(texture_finestre)(STRUCT([TOP([CUBOID([3,0.8,0.1]),CUBOID([3,0.5,0.1])])]))

    ContornoPorta = TEXTURE(texture_finestre)(T(2)(spessore_muri)(STRUCT([ContornoPortaA,ContornoPortaB,ContornoPortaC])))

    PortaCompleta1 = T(1)(((lunghezza_pannello_porta_p1-2)/2.0)-2)(STRUCT([Porta,ContornoPorta,T([1,2])([6,spessore_muri])(R([1,2])(-PI)(ContornoPorta))]))

    PortaCompleta = STRUCT([PortaCompleta1, T([1,2,3])([((lunghezza_pannello_porta_p1-2)/2.0)+2.5,spessore_muri-0.2,0.6+4.8])(R([1,3])(PI)(Mensola))])

    return PortaCompleta

In [14]:
VIEW(getPorta(6,8))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1631B0> >

<table>
    <tr><td>![porta.png](imm/porta.png)</td>
    <td>![porta.png](imm/porta-2.png)</td></tr>
</table>

### Finestra del secondo piano
- concettualmente simile alla definizione della finestra precedente.

In [15]:
# Finistra del piano 2

def getFinestraSecondoPiano (lunghezza_pannello_finestra_p1, altezza_pannello_finestra_p1, numero_pannelli_p1):

    Finestra1 = STRUCT([
        INSR(PROD)([QUOTE([0,2,-2,(lunghezza_pannello_finestra_p1-2)/2.0]), QUOTE([0,spessore_muri]), QUOTE([3])]),
        INSR(PROD)([QUOTE([-2,2]), QUOTE([0,spessore_muri]), QUOTE([0.6,-1.8,0.6])])
        ])

    DimSpazioBianco4 = ((lunghezza_pannello_finestra_p1-2)/2.0)-2
    SpazioBianco4 = CUBOID([DimSpazioBianco4,spessore_muri,altezza_pannello_finestra_p1])

    SpazioBianco2Sopra = CUBOID([lunghezza_pannello_finestra_p1,spessore_muri,((altezza_pannello_finestra_p1-3)*(2.0/3.0))])
    SpazioBianco2Sotto = CUBOID([lunghezza_pannello_finestra_p1,spessore_muri,((altezza_pannello_finestra_p1-3)*(1.0/3.0))])

    Finestra = LEFT([Finestra1,CUBOID([DimSpazioBianco4,spessore_muri,3])])

    Vfinestra = [[0,0],[0,3-0.6*2],[2,3-0.6*2],[2,0]]
    FVfinestra = [[0,1],[1,2],[2,3],[3,0]]

    ContornoFinestra1 = STRUCT(MKPOLS((Vfinestra,FVfinestra)))
    #ContornoFinestraA = T([1,3,2])([2-0.05,0.6-0.05,0.3])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.1,0.1])(ContornoFinestra1))(0.1)))
    #ContornoFinestraB = T([1,3,2])([2-0.1,0.6-0.1,0.2])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.2,0.2])(ContornoFinestra1))(0.1)))
    ContornoFinestraC = T([1,3,2])([2-0.2,0.6-0.2,0.1])(R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.4,0.4])(ContornoFinestra1))(0.1)))

    #ContornoFinestra = T(2)(spessore_muri)(STRUCT([ContornoFinestraA,ContornoFinestraB,ContornoFinestraC]))
    ContornoFinestra = TEXTURE(texture_finestre)(T(2)(spessore_muri)(STRUCT([ContornoFinestraC])))

    FinestraCompleta1 = T(1)(((lunghezza_pannello_finestra_p1-2)/2.0)-2)(STRUCT([Finestra,ContornoFinestra]))

    Vetro = getVetro(2,1.8,0.05)

    FinestraCompleta2 = STRUCT([FinestraCompleta1, T([1,2,3])([(lunghezza_pannello_finestra_p1-2)/2.0,spessore_muri,0.6])(Vetro)])

    FinestraCompleta = T(3)(((altezza_pannello_finestra_p1-3)*(1/3.0)))(STRUCT([FinestraCompleta2, T(3)(3)(SpazioBianco2Sopra),T(3)(-((altezza_pannello_finestra_p1-3)*(1/3.0)))(SpazioBianco2Sotto)]))
    
    ElementoMuroPiano1 = FinestraCompleta
    
    MuroPrimoPiano = RIGHT([SpazioBianco4,RIGHT([COMP([STRUCT, NN(numero_pannelli_p1)])([ElementoMuroPiano1, T(1)(lunghezza_pannello_finestra_p1)]),SpazioBianco4])])

    return MuroPrimoPiano

In [16]:
VIEW(getFinestraSecondoPiano(6,4,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D143570> >

<table>
    <tr><td>![finestraP2.png](imm/finestraP21.png)</td>
    <td>![finestraP2.png](imm/finestraP21-2.png)</td></tr>
</table>

### Finestra a forma di arco
- getFinestraArco(x,y,z) restituisce un blocco di muro contenente una finestra a forma di arco di lunghezz x, spessore y e altezza z;
- per generare questa finestra è stata utilizzata la funzione DIFFERENCE;
- per evitare la creazione di archi non completi è stato inserito un if che verifica che l'altezza del muro sia almeno maggiore o uguale al raggio della finestra.


In [17]:
# Finestra del secondo piano a forma di arco

def getFinestraArco(lunghezza,spessore,altezza):
        
    # questo if impedisce che venga creato una finestra non completa nel caso in cui l'altezza passata come
    # parametro fosse minore del raggio dell'arco
    if (altezza<lunghezza*0.70/2.0):
        altezza = lunghezza/2.0*0.9
    
    Parete = CUBOID([spessore,lunghezza,altezza])
    
    diametro_arco = lunghezza*0.70
    Arco = T([1,2,3])([spessore*1.5,lunghezza/2.0,altezza*0.05])(R([1,3])(PI/2)(JOIN([T(2)(-diametro_arco*0.5)(getMapSemiarco(diametro_arco,0)),T(2)(-diametro_arco*0.5)(getMapSemiarco(diametro_arco,spessore*2))])))
    
    Finestra1 = DIFFERENCE([Parete,Arco])
    
    Pilastro = CUBOID([spessore,diametro_arco/8.0,altezza])
    
    Vetro = T(1)(spessore/2.0)(getVetro(0.05,altezza,lunghezza))
    
    Finestra = STRUCT([Finestra1,T(2)(lunghezza*0.3)(Pilastro),T(2)(lunghezza*0.7-diametro_arco/8.0)(Pilastro),Vetro])
    
    return Finestra

In [18]:
VIEW(getFinestraArco(8,0.3,4))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D143BD0> >

<table>
    <tr><td>![finestra_arco.png](imm/finestra_arco.png)</td>
    <td>![finestra_arco.png](imm/finestra_arco-2.png)</td></tr>
</table>

### Angolari dell'edificio
- getAngolari(latoA,latoB,a) creare degli angolari posizionati sugli angoli di un rettangolo di dimensioni latoA x latoB e altezza a come quelli in figura.

In [19]:
# Ritorna 4 angolari che chiudono un rettangolo di dimensione latoAXlatoB

def getAngolari(latoA,latoB,altezza):
    
    AngolareMattoni = UNION([(STRUCT([T([1,2,3])([-0.2,-0.2,altezza])(getMattoniLaterali2()),T(2)(-0.2)(R([1,2])(PI/2)(T([1,2,3])([0,0,altezza])(getMattoniLaterali2())))]))])
    Angolari4 = STRUCT([AngolareMattoni,
                    T([1,2])([latoA+DimSpazioBianco*2,0])(R([1,2])(PI/2)(AngolareMattoni)),
                    T([1,2])([0,latoB+DimSpazioBianco*2])(R([1,2])(-PI/2)(AngolareMattoni)),
                    T([2,1])([latoB+DimSpazioBianco*2,latoA+DimSpazioBianco*2])(R([1,2])(PI)(AngolareMattoni))])
    return TEXTURE(texture_pietraP0)(Angolari4)


In [20]:
VIEW(getAngolari(1,2,2))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D143750> >

<table>
    <tr><td>![angolari.png](imm/angolari.png)</td>
    <td>![angolari.png](imm/angolari-2.png)</td></tr>
</table>

### Piano 0
- sono state utilizzate le funzioni precedentemente definite per costruire il piano 0 della Villa;
- vengono creati dei muri chiamando le opportune funzioni, ruotati e traslati dove necessario, e combinati per creare il piano 0 dell'edificio;
- tutte le traslazioni sono parametriche;
- pavimento e soffitto sono stati creati in modo indipendente per permettere di utilizzare texture differenti per i due elementi;
- sono stati realizzati dei fori su pavimento e soffitto per creare dei varchi per le scale.

In [21]:
# Piano 0

MuroLaterale = getFinestrella(dimensione_pannello_finestra,numero_finestre_laterali)

DueMuriLaterali = STRUCT([T([2,3])([spessore_muri,3])(R([2,3])(PI)(MuroLaterale)), T(2)(dimensione_lato_frontale_edificio+2*DimSpazioBianco-spessore_muri)(MuroLaterale)])

#MuroPosteriore = T(1)(spessore_muri)(R([1,2])(PI/2)(getFinestrella(dimensione_pannello_finestra,numero_finestre_posteriori)))
ElementoMuroPosteriore = T(1)(spessore_muri)(R([1,2])(PI/2)(getFinestrella(dimensione_pannello_finestra,2)))
MuroPosteriore = STRUCT([ElementoMuroPosteriore, T(2)(dimensione_pannello_finestra*4.0)(ElementoMuroPosteriore)])

ElementoMuroFrontale = getFinestrella(dimensione_pannello_finestra,1)

MuroFrontale = T([1,2,3])([dimensione_laterale_edificio*2.0+DimSpazioBianco*2-spessore_muri,-DimSpazioBianco*2-(-dimensione_pannello_finestra+8),3])(R([1,3])(PI)(T(1)(dimensione_laterale_edificio)(T(2)(2)(COMP([STRUCT, NN(2)])([R([1,2])(PI/2)(ElementoMuroFrontale), T(2)(dimensione_lato_frontale_edificio-(dimensione_lato_frontale_edificio*1.0/numero_finestre_posteriori))])))))

PerimetroPiano0 = STRUCT([DueMuriLaterali, MuroPosteriore, MuroFrontale])

# Base Piano 0
PavimentoP0 = CUBOID([dimensione_laterale_edificio+DimSpazioBianco*2.0+0.1,dimensione_lato_frontale_edificio+DimSpazioBianco*2.0+0.1,spessore_muri])

# Tetto Piano 0
SoffittoP0 = TEXTURE(texture_pietra)(TOP([T([1,2,3])([-0.25,-0.25,3])((CUBOID([dimensione_laterale_edificio+0.5+DimSpazioBianco*2.0,dimensione_lato_frontale_edificio+0.5+DimSpazioBianco*2.0,0.3]))), T([1,2])([-0.1,-0.1])(CUBOID([dimensione_laterale_edificio+0.2+DimSpazioBianco*2.0,dimensione_lato_frontale_edificio+0.2+DimSpazioBianco*2.0,spessore_muri*2]))]))

# Foro per scale tra piano 0 e primo piano
ForoScalePO = COLOR(RED)(T([1,2,3])([dimensione_pannello_finestra,dimensione_pannello_finestra*2.0+DimSpazioBianco-4,3])(CUBOID([dimensione_pannello_finestra+DimSpazioBianco*2.0,4,0.3+spessore_muri*2])))
SoffittoP0 = DIFFERENCE([SoffittoP0,ForoScalePO])
SoffittoP0 = TEXTURE(texture_pietra)(DIFFERENCE([SoffittoP0,T(2)((dimensione_lato_frontale_edificio+DimSpazioBianco*2.0)-(dimensione_pannello_finestra*4.0-DimSpazioBianco*2.0-spessore_muri))(ForoScalePO)]))

# Muro davanti le scale - piano 0
MuroDavantiScale0 = T([1,2])([dimensione_laterale_edificio+DimSpazioBianco*2.0-0.2,dimensione_lato_frontale_edificio*1.0/numero_finestre_posteriori])((TEXTURE(texture_pietraP0)(CUBOID([0.2,dimensione_lato_frontale_edificio-2*dimensione_pannello_finestra,3]))))

Angolari4 = getAngolari(dimensione_laterale_edificio,dimensione_lato_frontale_edificio,2)

Piano01 = STRUCT([PerimetroPiano0, MuroDavantiScale0, Angolari4])

Piano0 = TOP([PavimentoP0,TOP([Piano01,SoffittoP0])])

In [22]:
VIEW(Piano0)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D15F6F0> >

In [33]:
VIEW(STRUCT([TOP([Prato,Piano0])]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1F5C00> >

<table>
    <tr><td>![P0.png](imm/P0-3.png)</td>
    <td>![P0.png](imm/P0-4.png)</td></tr>
    <tr><td>![P0.png](imm/P0.png)</td>
    <td>![P0.png](imm/P0-2.png)</td></tr>
</table>

### Piano 1
- sono state utilizzate le funzioni precedentemente definite per costruire il piano 0 della Villa;
- vengono creati dei muri chiamando le opportune funzioni, ruotati e traslati dove necessario, e combinati per creare il piano 0 dell'edificio;
- tutte le traslazioni sono parametriche;
- pavimento e soffitto sono stati creati in modo indipendente per permettere di utilizzare texture differenti per i due elementi;
- sono stati realizzati dei fori su pavimento e soffitto per creare dei varchi per le scale.

In [24]:
# Piano 1

SpazioBianco4 = CUBOID([DimSpazioBianco,spessore_muri,altezza_primo_piano])

MuroLateraleP1 = getFinestraPrimoPiano(dimensione_pannello_finestra,altezza_primo_piano,numero_finestre_laterali)

DueMuriLateraliP1 = STRUCT([T([2,1])([spessore_muri,dimensione_laterale_edificio+2.0*DimSpazioBianco])(R([1,2])(PI)(MuroLateraleP1)), T([3,2])([0,dimensione_lato_frontale_edificio+DimSpazioBianco*2.0-spessore_muri])(MuroLateraleP1)])

#MuroPosterioreP1 = T(1)(spessore_muri)(R([1,2])(-3*PI/2)(getFinestraPrimoPiano(dimensione_pannello_finestra,altezza_primo_piano,numero_finestre_posteriori)))
ElementoMuroPosterioreP1 = T(1)(spessore_muri)(R([1,2])(-3*PI/2)(getFinestraPrimoPiano(dimensione_pannello_finestra,altezza_primo_piano,2)))
MuroPosterioreP1 = STRUCT([ElementoMuroPosterioreP1, T(2)(dimensione_pannello_finestra*4.0)(ElementoMuroPosterioreP1)])

ElementoMuroP1 = getFinestraPrimoPiano(dimensione_pannello_finestra,altezza_primo_piano,1)
MuroFrontaleP1 = T([1,2])([dimensione_laterale_edificio+2.0*DimSpazioBianco-spessore_muri,dimensione_pannello_finestra+2.0*DimSpazioBianco])(COMP([STRUCT, NN(2)])([R([1,2])(-PI/2)(ElementoMuroP1), T(2)(dimensione_lato_frontale_edificio-dimensione_pannello_finestra)]))

SoffittoP1 = CUBOID([dimensione_laterale_edificio+DimSpazioBianco*2,dimensione_lato_frontale_edificio+DimSpazioBianco*2.0,spessore_muri])
ForoPerSoffitto = COLOR(RED)(T([1,2])([spessore_muri,dimensione_pannello_finestra*2.0+DimSpazioBianco])(CUBOID([dimensione_laterale_edificio-dimensione_pannello_finestra*2.0,dimensione_lato_frontale_edificio-dimensione_pannello_finestra*4.0,spessore_muri])))
SoffittoP1 = DIFFERENCE([SoffittoP1,ForoPerSoffitto])

# Foro per scale tra piano 0 e primo piano
ForoScaleP1 = COLOR(RED)(T([1,2])([dimensione_pannello_finestra,dimensione_pannello_finestra*2.0+DimSpazioBianco-4])(CUBOID([dimensione_pannello_finestra+DimSpazioBianco*2.0,4,0.3+spessore_muri*2])))
PavimentoP1 = CUBOID([dimensione_laterale_edificio+DimSpazioBianco*2,dimensione_lato_frontale_edificio+DimSpazioBianco*2.0,0.1])
PavimentoP1 = DIFFERENCE([PavimentoP1,ForoScaleP1])
PavimentoP1 = TEXTURE(texture_pavimentoP1)(DIFFERENCE([PavimentoP1,T(2)((dimensione_lato_frontale_edificio+DimSpazioBianco*2.0)-(dimensione_pannello_finestra*4.0-DimSpazioBianco*2.0-spessore_muri))(ForoScaleP1)]))


Piano11 = STRUCT([DueMuriLateraliP1, MuroPosterioreP1, MuroFrontaleP1])

Finestra = getFinestraPrimoPiano(6,altezza_primo_piano,1)
Porta = getPorta(6,altezza_primo_piano)
Retro = R([1,2])(-3*PI/2)(STRUCT([Finestra,T(1)(4)(Porta),T(1)(8)(Finestra)]))

Piano1 = STRUCT([TOP([PavimentoP1,TOP([Piano11,SoffittoP1])]),T([1,2,3])([spessore_muri,(dimensione_pannello_finestra+DimSpazioBianco)*2.0,0.1])(Retro)])

In [25]:
VIEW(Piano1)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D15FE70> >

In [34]:
VIEW(STRUCT([TOP([TOP([Prato,Piano0]),Piano1])]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1F5BD0> >

<table>
    <tr><td>![P1.png](imm/P1a.png)</td>
    <td>![P1.png](imm/P1-2a.png)</td></tr>
    <tr><td>![P1.png](imm/P1-3a.png)</td>
    <td>![P1.png](imm/P1-4a.png)</td></tr>
</table>

### Piano 2
- sono state utilizzate le funzioni precedentemente definite per costruire il piano 0 della Villa;
- vengono creati dei muri chiamando le opportune funzioni, ruotati e traslati dove necessario, e combinati per creare il piano 0 dell'edificio;
- tutte le traslazioni sono parametriche;
- per quanto riguarda l'altezza del secondo piano ho deciso che la parte più alta del piano avesse un'altezza pari al 60% dell'altezza del primo piano e la parte più bassa alta il 40% dell'altezza del primo piano, cercando di ottenere un risultato più possibile simile alla Villa originale;
- pavimento e soffitto sono stati creati in modo indipendente per permettere di utilizzare texture differenti per i due elementi;
- sono stati realizzati dei fori su pavimento e soffitto per creare dei varchi per le scale.

In [27]:
# Piano 2

DimSpazioBianco5 = ((dimensione_pannello_finestra-2)/2.0)-2
SpazioBianco5 = CUBOID([DimSpazioBianco,spessore_muri,altezza_primo_piano])

MuroLateraleP2 = getFinestraSecondoPiano(dimensione_pannello_finestra,altezza_primo_piano*0.4,numero_finestre_laterali-1)

DueMuriLateraliP2 = STRUCT([T([2,1])([spessore_muri,(dimensione_laterale_edificio+2.0*DimSpazioBianco5)-dimensione_pannello_finestra])(R([1,2])(PI)(MuroLateraleP2)),
                            T([3,2])([0,dimensione_lato_frontale_edificio+DimSpazioBianco5*2.0-spessore_muri])(MuroLateraleP2)])

#MuroPosterioreP2 = T(1)(spessore_muri)(R([1,2])(-3*PI/2)(getFinestraSecondoPiano(dimensione_pannello_finestra,altezza_primo_piano*0.4,numero_finestre_posteriori)))
ElementoMuroPosterioreP2 = T(1)(spessore_muri)(R([1,2])(-3*PI/2)(getFinestraSecondoPiano(dimensione_pannello_finestra,altezza_primo_piano*0.4,2)))
MuroPosterioreP2 = STRUCT([ElementoMuroPosterioreP2, T(2)(dimensione_pannello_finestra*4.0)(ElementoMuroPosterioreP2)])

ElementoMuroP2 = getFinestraSecondoPiano(dimensione_pannello_finestra,altezza_primo_piano,1)
ElementoMuroP1 = getFinestraPrimoPiano(dimensione_pannello_finestra,altezza_primo_piano*0.6,1)

Tramezzo = T([2,1])([dimensione_pannello_finestra+DimSpazioBianco5*2.0-spessore_muri,dimensione_laterale_edificio-dimensione_pannello_finestra])(CUBOID([dimensione_pannello_finestra+DimSpazioBianco5*2,spessore_muri,altezza_primo_piano*0.6+spessore_muri]))
DueTramezzi = STRUCT([Tramezzo,T(2)(dimensione_laterale_edificio-DimSpazioBianco5*2.0+spessore_muri)(Tramezzo)])

MuroFrontaleP21 = T([1,2])([dimensione_laterale_edificio+2.0*DimSpazioBianco5-spessore_muri,dimensione_pannello_finestra+2.0*DimSpazioBianco5])(COMP([STRUCT, NN(2)])([R([1,2])(-PI/2)(ElementoMuroP1), T(2)(dimensione_lato_frontale_edificio-dimensione_pannello_finestra)]))

MuroFrontaleP2 = STRUCT([MuroFrontaleP21,
                         T([1,2])([dimensione_laterale_edificio-dimensione_pannello_finestra,dimensione_lato_frontale_edificio+DimSpazioBianco5*2.0-spessore_muri])(ElementoMuroP1),
                         T([1,2])([dimensione_laterale_edificio+dimensione_pannello_finestra-DimSpazioBianco5*6,spessore_muri])(R([1,2])(PI)(ElementoMuroP1))])

Arco = getFinestraArco(dimensione_lato_frontale_edificio-dimensione_pannello_finestra*4.0-DimSpazioBianco5*2.0,spessore_muri,altezza_primo_piano*0.4)
FinestraArco = T(2)((dimensione_pannello_finestra+DimSpazioBianco)*2.0)(Arco)

PavimentoP2 = CUBOID([dimensione_laterale_edificio+DimSpazioBianco*2,dimensione_lato_frontale_edificio+DimSpazioBianco*2.0,0.1])
ForoPerPavimento = COLOR(RED)(T([1,2])([spessore_muri,dimensione_pannello_finestra*2.0+DimSpazioBianco])(CUBOID([dimensione_laterale_edificio-dimensione_pannello_finestra*2.0,dimensione_lato_frontale_edificio-dimensione_pannello_finestra*4.0,spessore_muri])))
PavimentoP2 = TEXTURE(texture_pavimentoP2)(DIFFERENCE([PavimentoP2,ForoPerPavimento]))

SoffittoP2_1 = T(3)(altezza_primo_piano*0.4)(CUBOID([dimensione_laterale_edificio-dimensione_pannello_finestra,dimensione_lato_frontale_edificio+DimSpazioBianco*2.0,spessore_muri]))
SoffittoP2_2 = T([1,2,3])([dimensione_laterale_edificio-dimensione_pannello_finestra,0,altezza_primo_piano*0.6])(CUBOID([dimensione_pannello_finestra+DimSpazioBianco*2.0,dimensione_pannello_finestra+DimSpazioBianco*2.0,spessore_muri]))
SoffittoP2_3 = T(2)(dimensione_lato_frontale_edificio-dimensione_pannello_finestra)(SoffittoP2_2)
#SoffittoP2_4 = T([1,3])([dimensione_laterale_edificio-dimensione_pannello_finestra,altezza_primo_piano*0.4])(CUBOID([spessore_muri,dimensione_lato_frontale_edificio+DimSpazioBianco5*2.0,altezza_primo_piano*0.6-altezza_primo_piano*0.4+spessore_muri]))
SoffittoP2_41 = T([1,3])([dimensione_laterale_edificio-dimensione_pannello_finestra,altezza_primo_piano*0.4])(CUBOID([spessore_muri,dimensione_pannello_finestra+DimSpazioBianco*2.0,altezza_primo_piano*0.6-altezza_primo_piano*0.4+spessore_muri]))
SoffittoP2_4 = STRUCT([SoffittoP2_41, T(2)(dimensione_laterale_edificio+dimensione_pannello_finestra)(SoffittoP2_41)])
SoffittoP2_5 = T([1,2])([dimensione_laterale_edificio-dimensione_pannello_finestra,dimensione_pannello_finestra+DimSpazioBianco*2])(CUBOID([spessore_muri,dimensione_lato_frontale_edificio-dimensione_pannello_finestra*2.0-DimSpazioBianco*2.0,altezza_primo_piano*0.4+spessore_muri]))

SoffittoP2 = STRUCT([SoffittoP2_1,SoffittoP2_2,SoffittoP2_3,SoffittoP2_4,SoffittoP2_5])


Piano2 = STRUCT([PavimentoP2,DueMuriLateraliP2, MuroPosterioreP2, MuroFrontaleP2, SoffittoP2, DueTramezzi, FinestraArco])

In [28]:
VIEW(Piano2)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1F52A0> >

In [35]:
VIEW(STRUCT([TOP([TOP([Prato,Piano0]),Piano1]),
             T([1,2,3])([posizione_piano2_X,posizione_piano2_Y,altezza_secondo_piano+0.1])(Piano2)
            ]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1F53F0> >

<table>
    <tr><td>![P2.png](imm/P2.png)</td>
    <td>![P2.png](imm/P2-2.png)</td></tr>
    <tr><td>![P2.png](imm/P2-3a.png)</td>
    <td>![P2.png](imm/P2-4a.png)</td></tr>
</table>

### Scalinata frontale
- getScalinata(r,a) restituisce una scalinata di 6 gradini di raggio r ed altezza totale a;
- sono state utilizzate le funzioni di scalo e le traslazioni per realizzare la scalinata

In [30]:
# Scalinata frontale di 6 gradini

def Scalino(r,altezza):
    elemento = JOIN([T(2)(-r*0.5)(getMapSemiarco(r,0)),T(2)(-r*0.5)(getMapSemiarco(r,altezza))])
    
    return elemento

def getScalinata(r,altezza_totale):
    
    altezza_scalino = altezza_totale/6.0
    
    scalino_base = Scalino(r,altezza_scalino)
    
    Scalinata1 = T(3)(altezza_scalino*5.0)(STRUCT([
                        scalino_base,
                        T(3)(-altezza_scalino)(S([1,2])([1.15,1.15])(scalino_base)),
                        T(3)(-2*altezza_scalino)(S([1,2])([1.3,1.3])(scalino_base)),
                        T(3)(-3*altezza_scalino)(S([1,2])([1.45,1.45])(scalino_base)),
                        T(3)(-4*altezza_scalino)(S([1,2])([1.6,1.6])(scalino_base)),
                        T(3)(-5*altezza_scalino)(S([1,2])([1.75,1.75])(scalino_base)),
                        ]))

    Scalinata = TEXTURE(texture_pietra)(Scalinata1)
    
    return Scalinata

Scalinata = getScalinata(raggio_scalinata_frontale,altezza_scalinata_frontale)

In [31]:
VIEW(getScalinata(8,2))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1F59F0> >

<table>
    <tr><td>![angolari.png](imm/scalinata.png)</td>
    <td>![angolari.png](imm/scalinata-2.png)</td></tr>
</table>

In [32]:
VIEW(STRUCT([TOP([TOP([Prato,Piano0]),Piano1]),
             T([1,2,3])([posizione_piano2_X,posizione_piano2_Y,altezza_secondo_piano+0.1])(Piano2),
             T([1,2,3])([posizione_facciata_Y+spessore_facciata,dim_prato_y/2.0,altezza_piano_0-altezza_scalinata_frontale])(Scalinata)]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1F5450> >

### Scalinata posteriore
- getScalinataPosteriore(a,b,c,d,e,f,g) restituisce una scalinata come quella in figura di altezza a, larghezza b, numero_gradini c, profondita_scala d, lunghezza_cima e, altezza bordo scalinata f e altezza dei bordi in cima destra e sinistra rispettivamente f e g;
- "lunghezza cima" indica la lunghezza del piano in cima alle scale;
- per realizzarla è stata creata un funzione che restituisce un singolo gradino all'interno di un ciclo, con i parametri opportunamente variati;
- la scalinata occupa il 90% della lunghezza passata come parametro. Il restante 20% è occupato dalle decorazioni laterali;
- le decorazioni laterali sono state crete utilizzando la funzione JOIN;
- Questo tipo di scala è stata usata sia per la scalinata esterna posteriore che per le scalinate interne all'edificio.

In [36]:
# Scalinata posteriore di 12 gradini

def getScalinoPosteriore(l,altezza,profondita):
    elemento = CUBOID([l,profondita,altezza])
    return elemento

def getScalinataPosteriore(altezza,larghezza,numero_gradini,profondita_scala,lunghezza_cima,altezza_bordo,altezza_cima_dx,altezza_cima_sx):
        
    profondita_gradino = profondita_scala*1.0/numero_gradini
    altezza_scalino = altezza*1.0/numero_gradini
    
    profondita_scala = numero_gradini*profondita_gradino
    scalini = [T(3)(altezza_scalino*i)(getScalinoPosteriore(larghezza*0.8,altezza_scalino,profondita_scala-(profondita_gradino*i))) for i in np.arange(numero_gradini)]
    rampa = [T(3)(altezza_scalino*i)(getScalinoPosteriore(larghezza*0.1,altezza_bordo+altezza_scalino,profondita_scala-(profondita_gradino*i))) for i in np.arange(numero_gradini)]
    
    scalini = STRUCT(scalini)
    rampa = JOIN(STRUCT(rampa))
    
    cima1 = CUBOID([larghezza*0.8,lunghezza_cima,altezza])
    cima_bordo_sx = CUBOID([larghezza*0.1,lunghezza_cima,altezza+altezza_cima_sx])
    cima_bordo_dx = CUBOID([larghezza*0.1,lunghezza_cima,altezza+altezza_cima_dx])
    
    cima = STRUCT([RIGHT([cima_bordo_sx,RIGHT([cima1,cima_bordo_dx])])])
    
    scalinata1 = STRUCT([RIGHT([rampa,RIGHT([scalini,rampa])])])
    scalinata = T(2)(-larghezza/2.0)(R([1,2])(PI/2)(STRUCT([UP([cima,scalinata1])])))
    return scalinata

Scalinata_posteriore = getScalinataPosteriore(altezza_piano_0,dimensione_pannello_finestra*2.0-DimSpazioBianco*2.0,13,6.5,2,1,1,1)

In [38]:
VIEW(getScalinataPosteriore(20,10,20,30,5,1,1,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1C9900> >

<table>
    <tr><td>![angolari.png](imm/scalinata_posteriore.png)</td>
    <td>![angolari.png](imm/scalinata_posteriore-2.png)</td></tr>
</table>

In [39]:
VIEW(STRUCT([TOP([TOP([Prato,Piano0]),Piano1]),
             T([1,2,3])([posizione_piano2_X,posizione_piano2_Y,altezza_secondo_piano+0.1])(Piano2),
             T([1,2,3])([posizione_facciata_Y+spessore_facciata,dim_prato_y/2.0,altezza_piano_0-altezza_scalinata_frontale])(Scalinata),
             T([1,2])([posizione_piano2_X,dim_prato_y/2.0])(Scalinata_posteriore)]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1C99C0> >

# Definizione dei metodi per creare la facciata della Villa

### Colonne di mattoncini
- getColonneMattoncini(a,b,c,d,e,f) restituisce delle colonne di mattoncini come quelle in figura di spessore a, larghezza b, distanza_tra_colonne c, altezza_colonna d e numero_mattoncini e;
- ho scelto di procedere in questo modo per avere anche in questo caso una funzione parametrica e per ricreare l'effetto dei mattoncini presente nella villa originale;
- ho preferito creare i mattoncini invece che utilizzare semplicemente una texture

In [40]:
# Colonne di Mattoncini

def getColonneMattoncini(spessore,larghezza,distanza_tra_colonne,altezza_colonna,numero_mattoncini):
    
    altezza_mattoncino = (altezza_colonna*1.0/numero_mattoncini)-0.15

    Mattoncino = CUBOID([spessore,larghezza,altezza_mattoncino])
    
    BaseColonnaMattoncini = CUBOID([spessore-0.1,larghezza,altezza_colonna])
    
    ColonnaMattoncini1 = COMP([STRUCT, NN(numero_mattoncini)])([Mattoncino, T(3)(altezza_mattoncino+0.15)])

    ColonnaMattoncini = UNION([STRUCT([BaseColonnaMattoncini,ColonnaMattoncini1])])

    ColonnaMattoncini4 = COMP([STRUCT, NN(4)])([ColonnaMattoncini, T(2)(distanza_tra_colonne)])

    ColonneMattonciniFinite = TEXTURE(texture_pietra)(STRUCT([ColonnaMattoncini,T(2)(larghezza+0.16)(ColonnaMattoncini4),T(2)(distanza_tra_colonne*3.0+larghezza*2.0+0.16*2)(ColonnaMattoncini)]))
    
    return ColonneMattonciniFinite

In [41]:
VIEW(getColonneMattoncini(3,1.6,5.8,8,22))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1C9E40> >

<table>
    <tr><td>![colonne.png](imm/colonne_mattoni.png)</td>
    <td>![colonne.png](imm/colonne_mattoni-2.png)</td></tr>
</table>

### Colonne sotto gli archi
- restituisce il colonnato presente sotto gli archi

In [42]:
# Colonne sotto archi

def getColonna(w,h,s):
    basis = CUBOID([w, w, w/ 6.0])
    trunk = CYLINDER([w / 2.0*0.85, h-(2.0*w/6.0)])(24)
    capitel = CUBOID([w, w*1.5+s, w/6.0])
    return TOP([TOP([basis, trunk]), capitel])


def getColonnato(dm,h,numero_colonne,spazio):
    colonnato = STRUCT(NN(numero_colonne)([getColonna(dm,h,spazio),T(2)(spazio+dm)]))
    return TEXTURE(texture_pietra)(colonnato)

In [43]:
VIEW(getColonnato(0.4,1.2,6,0.6))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1C9B10> >

<table>
    <tr><td>![colonnato.png](imm/colonnato.png)</td>
    <td>![colonnato.png](imm/colonnato-2.png)</td></tr>
</table>

### Archi della facciata
- getArchi(a,b,c,d,e) restituisce una fila di archi di lunghezza a, altezza b, profondità c, numero_archi d e spazio_tra_archi e;
- sono state utilizzate, dopo averle opportunatamente modificate, delle funzioni presenti in uno dei capitoli del libro studiato.

In [44]:
# Arco di mattoni

def ArchSurface(rr,w,h):
    x1 = lambda k: rr*cos(k[0])
    y1 = lambda k: rr*sin(k[0])
    x2 = lambda k: (rr-w)*cos(k[0])
    y2 = lambda k: (rr-w)*sin(k[0])
    z = lambda k: h
    
    Circle0 = BEZIER(S1)([CONS([x1,y1,z])])
    
    Circle1 = BEZIER(S1)([CONS([x2,y2,z])])
    
    return BEZIER(S2)([Circle0, Circle1])

def getArco1(length,w,depth):
    def Arch1(angle):
        
        radius=(length/2)/cos(angle/2)
        
        domain2D=PROD([T(1)(angle/2)(INTERVALS(PI-angle)(13)),QUOTE([1])])
        
        domain3D=PROD([domain2D, QUOTE([1])])
        
        ArchSurf2D_0=ArchSurface(radius,w,0)
        
        ArchSurf2D_1=ArchSurface(radius,w,depth)
        
        SolidMap=BEZIER(S3)([ArchSurf2D_0, ArchSurf2D_1])
        
        return MAP(SolidMap)(domain3D)
    
    return Arch1

def getArco(lunghezza,altezza,profondita):
    
    Arco = R([1,2])(PI/2)(R([2,3])(-3*PI/2)(getArco1(lunghezza, altezza, profondita)(0.001*PI)))
    
    return Arco

def getArchi(lunghezza,altezza,profondita,numero_archi,spazio_tra_archi):
    
    Archi = T(2)(lunghezza/2.0)(STRUCT(NN(numero_archi)([getArco(lunghezza,altezza,profondita),T(2)(spazio_tra_archi+lunghezza)])))
    
    return TEXTURE(texture_pietra)(Archi)

In [45]:
VIEW(getArchi(10,2,1.6,3,-2))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1EE570> >

<table>
    <tr><td>![archi.png](imm/archi.png)</td>
    <td>![archi.png](imm/archi-2.png)</td></tr>
</table>

### Fregio sopra gli archi
- getCornice(a,b,c,d,e) restituisce un fregio decorativo di a numero di pannelli, di spessore b, larghezza c, altezza della parte decorata e e della parte lisca d;
- anche in uqesto caso la funzione è parametrica e sfrutta l'utilizzo delle funzioni COMP e NN.

In [46]:
# Elemento sopra gli archi

def getCornice(numeroElementiPannello,spessore,larghezza,altezza_base,altezza_decorazione):
    
    larghezza_pannello = (larghezza*1.0/numeroElementiPannello)*0.7
    
    larghezza_decorazione = ((larghezza*1.0/numeroElementiPannello)*0.3)/4.0

    ElementoPannello1 = COMP([STRUCT, NN(4)])([CUBOID([spessore,larghezza_decorazione,altezza_decorazione]), T(2)(larghezza_decorazione*1.2)])
    
    BaseElementoPannello = CUBOID([spessore-0.1,larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,altezza_decorazione])

    ElementoPannello = STRUCT ([ElementoPannello1,BaseElementoPannello])

    ElementoPannello3 = CUBOID([spessore-0.1,larghezza_pannello,altezza_decorazione]) # cubo tra elementi

    ElementoPannello4 = STRUCT([ElementoPannello,
                        T(2)(larghezza_decorazione*1.2*4-larghezza_decorazione*0.2)(ElementoPannello3),
                        T(3)(altezza_decorazione)(CUBOID([spessore,larghezza_pannello+larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,0.2])),
                        CUBOID([spessore,larghezza_pannello+larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,0.2]),
                        T(3)(-altezza_base)(CUBOID([spessore-0.1,larghezza_pannello+larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,altezza_base]))
                        ])

    ElementoUltimoPannello = STRUCT([ElementoPannello,
                            T(3)(altezza_decorazione)(CUBOID([spessore,larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,0.2])),
                            CUBOID([spessore,larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,0.2]),
                            T(3)(-altezza_base)(CUBOID([spessore-0.1,larghezza_decorazione*1.2*4-larghezza_decorazione*0.2,altezza_base]))
                        ])

    ElementoPannelloCompleto1 = COMP([STRUCT, NN(numeroElementiPannello)])([ElementoPannello4, T(2)(larghezza_pannello+larghezza_decorazione*1.2*4-larghezza_decorazione*0.2)])
    ElementoPannelloCompleto = T(3)(altezza_base)(STRUCT([ElementoPannelloCompleto1, T(2)(((larghezza_decorazione*1.2*4-larghezza_decorazione*0.2)+larghezza_pannello)*numeroElementiPannello)(ElementoUltimoPannello)]))
    
    return TEXTURE(texture_pietra)(UNION([ElementoPannelloCompleto]))

In [47]:
VIEW(getCornice(8,1.6,10,1,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1EE720> >

<table>
    <tr><td>![fregio.png](imm/fregio.png)</td>
    <td>![fregio.png](imm/fregio-2.png)</td></tr>
</table>

### Frontone della facciata
- getTriangolo(lunghezza,altezza,spessore) restituisce un frontone delle dimensioni specificate nei parametri della funzione;
- per realizzarlo ho utilizzato anche le funzioni MKPOLS e MULTEXTRUDE.

In [48]:
# Trinagolo su archi

def getTriangolo(lunghezza,altezza,spessore):
    punti = [[0,0],[0,lunghezza],[altezza,lunghezza/2.0]]
    P = AA(MK)(punti)
    S = AA(JOIN)([P[0],P[1],P[2]])
    H = JOIN(S)
    Triangolo = R([2,3])(PI/2)(MULTEXTRUDE(H)(1.6))

    V = [[0,0],[0,0.5],[0.5,0.5],[0.5,1],[1,1],[1,0.5],[1.5,0.5],[1.5,0]]
    FV = [[0,1,3,7],[2,3,4,7],[4,5,6,7]]

    A = STRUCT(MKPOLS((V,FV)))

    V1 = [[0,0],[0,lunghezza],[altezza,lunghezza/2.0]]
    FV1 = [[0,1],[1,2],[0,2]]

    ContornoTriangolo1 = STRUCT(MKPOLS((V1,FV1)))
    ContornoTriangolo = R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([1,0])(ContornoTriangolo1))(2))
    ContornoTriangoloGrande = R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([0.2,0])(ContornoTriangolo1))(2.2))
    ContornoTriangoloDietro = R([2,3])(PI/2)(MULTEXTRUDE(OFFSET([2,1])(ContornoTriangolo1))(spessore))

    TriangoloCompleto = T([1,2])([spessore,0.5])(R([2,3])(-PI/2)(R([1,2])(-PI/2)(STRUCT([Triangolo, 
                                                                       T(2)(0.8)(ContornoTriangolo),
                                                                       T([2,1])([1.2,0.5])(ContornoTriangoloGrande),
                                                                       T(3)(-0.5)(ContornoTriangoloDietro)
                                                                      ]))))
    
    return TEXTURE(texture_pareti)(UNION([TriangoloCompleto]))

In [49]:
VIEW(getTriangolo(26,5,1.6))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1EEEA0> >

<table>
    <tr><td>![frontone.png](imm/frontone.png)</td>
    <td>![frontone.png](imm/frontone-2.png)</td></tr>
</table>

### Costruzione della facciata
- Qui viene costruite la facciata della Villa utilizzando le funzioni precedentemente definite

In [50]:
# Definizione facciata frontale

ColonneMattoncini = TEXTURE(texture_pietra)(getColonneMattoncini(spessore_facciata,2,7.9,12,14))

Archi = TEXTURE(texture_pietra)(T([2,3])([2+0.16,7.5])(getArchi(8.3+1.6,2,spessore_facciata,3,-2)))

Cornicione = TEXTURE(texture_pietra)(T(3)(11)(getCornice(14,spessore_facciata,28.1,1.6,1.5)))

Triangolo = T(3)(11+1.6+1.5)(getTriangolo(29.05,7,spessore_facciata))

Colonnato = T([2,1,3])([4.24,spessore_facciata/2.0,0.2])(getColonnato(0.5,1.5,9,0.15))
Colonnato = STRUCT([Colonnato,T(2)(7.9*2.0)(Colonnato)])

Facciata = STRUCT([ColonneMattoncini, Archi, Cornicione, Triangolo, Colonnato])

In [51]:
VIEW(Facciata)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1EEF00> >

<table>
    <tr><td>![facciata.png](imm/facciata.png)</td>
    <td>![facciata.png](imm/facciata-2.png)</td></tr>
</table>

In [52]:
VIEW(STRUCT([TOP([TOP([Prato,Piano0]),Piano1]),
             T([1,2,3])([posizione_piano2_X,posizione_piano2_Y,altezza_secondo_piano+0.1])(Piano2),
             T([1,2,3])([posizione_facciata_Y+spessore_facciata,dim_prato_y/2.0,altezza_piano_0-altezza_scalinata_frontale+0.1])(Scalinata),
             T([1,2])([posizione_piano2_X,dim_prato_y/2.0])(Scalinata_posteriore),
             T([1,2,3])([posizione_facciata_Y,posizione_facciata_X,posizione_facciata_Z])(Facciata)]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1EEF90> >

<table>
    <tr><td>![facciata_completa.png](imm/facciata_completa.png)</td>
    <td>![facciata_completa.png](imm/facciata_completa-2.png)</td></tr>
    <tr><td>![facciata_completa.png](imm/facciata_completa-3.png)</td>
    <td>![facciata_completa.png](imm/facciata_completa-4.png)</td></tr>
</table>

# Definizione del tetto
- sono state definite tre funzioni per generare tre tipologie differenti di tetti;
- in tutti i casi è stato utilizzato MKPOLS dopo aver generato una lista di liste di coordinate opportunatamente definite per mezzo di parametri.

In [53]:
def getTetto1(lato,altezza):
    V1 = [[0,0,-altezza],[0,lato,-altezza],[lato,0,-altezza],[lato,lato,-altezza],[lato/2.0,lato/2.0]]
    FV1 = [[0,2,4],[2,3,4]]

    Tetto1 = STRUCT(MKPOLS((V1,FV1)))
    
    Tetto = T(3)(altezza)(STRUCT([Tetto1,R([1,2])(PI)(T([1,2])([-lato,-lato])(Tetto1))]))
    
    return TEXTURE(texture_tetto)(Tetto)

def getTetto2(latoA,latoB,altezza):
    
    V1 = [[0,0,-altezza],[0,latoA,-altezza],[latoB,0,-altezza],[latoB,latoA,-altezza],[latoB*0.35,latoA/2.0],[latoB*0.65,latoA/2.0]]

    FVa = [[2,3,5]]
    FVb = [[1,3],[4,5]]
   
    TettoA = STRUCT(MKPOLS((V1,FVa)))
    TettoB = JOIN(STRUCT(MKPOLS((V1,FVb))))
    
    Tetto1 = STRUCT([TettoA,TettoB])

    Tetto = T(3)(altezza)(STRUCT([Tetto1,R([1,2])(PI)(T([1,2])([-latoB,-latoA])(Tetto1))]))
    
    return TEXTURE(texture_tetto)(Tetto)

def getTetto3(lunghezza,altezza,spessore):
    
    V1 = [[0,0],[0,lunghezza],[altezza,lunghezza/2.0]]
    
    FV1 = [[1,2],[0,2]]
    
    TriangoloPerTetto = STRUCT(MKPOLS((V1,FV1)))
    
    Tetto = R([1,3])(PI/2)(MULTEXTRUDE(TriangoloPerTetto)(spessore))
    
    return TEXTURE(texture_tetto)(Tetto)

In [54]:
VIEW(getTetto1(4,2))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1DEF00> >

In [55]:
VIEW(getTetto2(6,8,2))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1DEF90> >

In [56]:
VIEW(getTetto3(10,2,4))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1DE540> >

<table>
    <tr><td>![tetto.png](imm/tetto.png)</td>
    <td>![tetto.png](imm/tetto-2.png)</td>
    <td>![tetto.png](imm/tetto-3.png)</td></tr>
</table>

### Costruzione del tetto
- sfruttando le funzioni precedenti è stato costruito il tetto della villa

In [57]:
# Tetto

lato = dimensione_pannello_finestra + DimSpazioBianco*2.0
TettoQuadrato = JOIN(getTetto1(lato, altezza_tetti))

#COMP([STRUCT, NN(2)])([R([1,2])(-PI/2)(ElementoMuroP1), T(2)(dimensione_lato_frontale_edificio-dimensione_pannello_finestra)]))

TettiQuadrati = TEXTURE(texture_tetto)(COMP([STRUCT, NN(2)])([TettoQuadrato,T(2)(dimensione_lato_frontale_edificio-dimensione_pannello_finestra)]))

TettoCentrale = TEXTURE(texture_tetto)(JOIN(R([1,2])(PI/2)(getTetto2(dimensione_laterale_edificio-dimensione_pannello_finestra,dimensione_lato_frontale_edificio+DimSpazioBianco*2.0,altezza_tetti*4))))

TettoTriangolo = JOIN(T([1,2])([lato,lato+0.5])(getTetto3(29.05,7,dimensione_laterale_edificio+DimSpazioBianco*2.0-(dimensione_laterale_edificio-dimensione_pannello_finestra)/2.0)))

TettoDietroTriangolo = TEXTURE(texture_tetto)(T([2,3])([dimensione_pannello_finestra+DimSpazioBianco*2.0,-(altezza_primo_piano*0.4+spessore_muri)])(CUBOID([spessore_muri*2.0,dimensione_lato_frontale_edificio-dimensione_pannello_finestra*2.0-DimSpazioBianco*2.0,altezza_primo_piano*0.4+spessore_muri])))


da_sottrarre1 = COLOR(GREEN)(JOIN(TettoCentrale))
#da_sottrarre2 = COLOR(RED)(CUBOID([dimensione_laterale_edificio-dimensione_pannello_finestra,dimensione_lato_frontale_edificio+DimSpazioBianco*2.0,7]))
#da_sottrarre = T([1,3])([-(dimensione_laterale_edificio-dimensione_pannello_finestra),-7])(STRUCT([TOP([da_sottrarre2,da_sottrarre1])]))

TettoTriangolo = TEXTURE(texture_tetto)(DIFFERENCE([TettoTriangolo,da_sottrarre1]))
#TettoTriangolo = DIFFERENCE([TettoTriangolo,da_sottrarre2])


Tetto1 = STRUCT([T(3)(altezza_primo_piano*0.6-altezza_primo_piano*0.4)(TettiQuadrati),
                T(1)(0)(TettoCentrale),
                T(3)(-altezza_primo_piano*0.4+1)(TettoTriangolo)])

Tetto = STRUCT([Tetto1,TettoDietroTriangolo])

In [58]:
VIEW(Tetto)

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1DEC00> >

<table>
    <tr><td>![tetto_completo.png](imm/tetto_completo.png)</td>
    <td>![tetto_completo.png](imm/tetto_completo-2.png)</td></tr>
</table>

In [59]:
VIEW(STRUCT([TOP([TOP([Prato,Piano0]),Piano1]),
             T([1,2,3])([posizione_piano2_X,posizione_piano2_Y,altezza_secondo_piano+0.1])(Piano2),
             T([1,2,3])([posizione_tetto_X,posizione_tetto_Y,altezza_tetto+0.1])(Tetto),
             T([1,2,3])([posizione_facciata_Y+spessore_facciata,dim_prato_y/2.0,altezza_piano_0-altezza_scalinata_frontale+0.1])(Scalinata),
             T([1,2])([posizione_piano2_X,dim_prato_y/2.0])(Scalinata_posteriore),
             T([1,2,3])([posizione_facciata_Y,posizione_facciata_X,posizione_facciata_Z])(Facciata)]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1DEBD0> >

<table>
    <tr><td>![villa_con_tetto.png](imm/villa_con_tetto.png)</td>
    <td>![villa_con_tetto.png](imm/villa_con_tetto-2.png)</td></tr>
    <tr><td>![villa_con_tetto.png](imm/villa_con_tetto-3.png)</td>
    <td>![villa_con_tetto.png](imm/villa_con_tetto-4.png)</td></tr>
</table>

### Costruzione interni della Villa
- le pareti interne sono state costruite tenendo conto delle dimensioni precedentemente specificate, come lo spessore dei muri, la posizione della villa, l'altezza dei piani ecc.
- con la funzione DIFFERENCE sono stati creati i varchi per le porte;
- sono state posizionate le scale necessarie ad accedere ai diversi piani della Villa.

In [60]:
# Muri interni
def getParetiInterne(dimensione_laterale_edificio,dimensione_lato_frontale_edificio,altezza):
    PareteP2_4 = CUBOID([spessore_muri,dimensione_lato_frontale_edificio+DimSpazioBianco5*2.0,altezza_secondo_piano+altezza_primo_piano*0.4+spessore_muri])
    Parete2 = T(2)([dimensione_pannello_finestra+DimSpazioBianco5*2.0-spessore_muri])(CUBOID([dimensione_pannello_finestra+DimSpazioBianco5*2,spessore_muri,altezza+spessore_muri]))
    DueParete2 = STRUCT([Parete2,T(2)(dimensione_laterale_edificio-DimSpazioBianco5*2.0+spessore_muri)(Parete2)])
    
    # riferimento per creare varchi per le porte
    foroPerPorta = CUBOID([spessore_muri,3,6])
    
    # porte primo piano
    PareteP2_4 = DIFFERENCE([PareteP2_4,T([2,3])([-1.5+(dimensione_lato_frontale_edificio+DimSpazioBianco*2.0)/2.0,altezza_piano_0])(foroPerPorta)])
    DueParete2 = DIFFERENCE([DueParete2,T([1,2,3])([dimensione_pannello_finestra/2.0,dimensione_pannello_finestra+DimSpazioBianco*2.0,altezza_piano_0])(R([1,2])(-PI/2)(foroPerPorta))])
    DueParete2 = DIFFERENCE([DueParete2,T([1,2,3])([dimensione_pannello_finestra/2.0,dimensione_lato_frontale_edificio-(dimensione_pannello_finestra-spessore_muri),altezza_piano_0])(R([1,2])(-PI/2)(foroPerPorta))])
    PareteP2_4 = DIFFERENCE([PareteP2_4,T([2,3])([-1.5+dimensione_pannello_finestra/2.0+DimSpazioBianco,altezza_piano_0])(foroPerPorta)])
    PareteP2_4 = DIFFERENCE([PareteP2_4,T([2,3])([dimensione_lato_frontale_edificio+1.5-dimensione_pannello_finestra/2.0-DimSpazioBianco,altezza_piano_0])(foroPerPorta)])
    
    PareteD = T([1,2])([-dimensione_pannello_finestra*3.0,dimensione_pannello_finestra*2.0+DimSpazioBianco-spessore_muri])(CUBOID([dimensione_pannello_finestra,spessore_muri,altezza_secondo_piano+altezza_primo_piano*0.4+spessore_muri]))
    DuePareteD = STRUCT([PareteD,T(2)(dimensione_pannello_finestra*2.0+spessore_muri)(PareteD)])
    
    PareteP2_5 = CUBOID([spessore_muri,dimensione_lato_frontale_edificio+DimSpazioBianco*2.0,altezza_secondo_piano+altezza_primo_piano*0.4+spessore_muri])
    PareteP2_5 = DIFFERENCE([PareteP2_5,T([2,3])([-1.5+(dimensione_lato_frontale_edificio+DimSpazioBianco*2.0)/2.0,altezza_piano_0])(foroPerPorta)])
    PareteP2_5 = DIFFERENCE([PareteP2_5,T([2,3])([-1.5+dimensione_pannello_finestra/2.0+DimSpazioBianco,altezza_piano_0])(foroPerPorta)])
    PareteP2_5 = DIFFERENCE([PareteP2_5,T([2,3])([dimensione_lato_frontale_edificio+1.5-dimensione_pannello_finestra/2.0-DimSpazioBianco,altezza_piano_0])(foroPerPorta)])
    
    # muretto vicino scale secondo piano
    MuroVicinoScale1 = T([1,2,3])([-dimensione_pannello_finestra,dimensione_pannello_finestra*2.0+DimSpazioBianco,altezza_secondo_piano])(CUBOID([spessore_muri,dimensione_pannello_finestra*2.0,3]))
    MuroVicinoScale21 = T([1,2,3])([-dimensione_pannello_finestra*2.0+4.0,dimensione_pannello_finestra*2.0+DimSpazioBianco,altezza_secondo_piano])(CUBOID([dimensione_pannello_finestra-4.0+spessore_muri,spessore_muri,3]))
    MuroVicinoScale22 = T([1,2,3])([-dimensione_pannello_finestra*2.0+4.0,(dimensione_lato_frontale_edificio+DimSpazioBianco*2.0)-(dimensione_pannello_finestra*2.0+DimSpazioBianco),altezza_secondo_piano])(CUBOID([dimensione_pannello_finestra-4.0+spessore_muri,spessore_muri,3]))
    MuroVicinoScale = STRUCT([MuroVicinoScale1,MuroVicinoScale21,MuroVicinoScale22])
    
    # porta principale primo piano
    Porta1 = T([2,3])([1.5+dimensione_lato_frontale_edificio/2.0+DimSpazioBianco*2.0,altezza_piano_0])(R([1,2])(-PI/2)(getPorta(6,6)))
    Porta2 = T([1,2,3])([-dimensione_pannello_finestra*2.0,1.5+dimensione_lato_frontale_edificio/2.0+DimSpazioBianco*2.0,altezza_piano_0])(R([1,2])(-PI/2)(getPorta(6,6)))
    Porta = STRUCT([Porta1,Porta2])
    
    # porte secondo piano
    PareteP2_4 = DIFFERENCE([PareteP2_4,T([2,3])([-1.5+dimensione_pannello_finestra/2.0+DimSpazioBianco,altezza_secondo_piano])(foroPerPorta)])
    PareteP2_4 = DIFFERENCE([PareteP2_4,T([2,3])([dimensione_lato_frontale_edificio+1.5-dimensione_pannello_finestra/2.0-DimSpazioBianco,altezza_secondo_piano])(foroPerPorta)])
    PareteP2_5 = DIFFERENCE([PareteP2_5,T([2,3])([-1.5+dimensione_pannello_finestra/2.0+DimSpazioBianco,altezza_secondo_piano])(foroPerPorta)])
    PareteP2_5 = DIFFERENCE([PareteP2_5,T([2,3])([dimensione_lato_frontale_edificio+1.5-dimensione_pannello_finestra/2.0-DimSpazioBianco,altezza_secondo_piano])(foroPerPorta)])
    
    # scalinate tra primo e secondo piano
    Scalinata1 = T([1,2,3])([-dimensione_pannello_finestra*2,2+dimensione_pannello_finestra*2.0+DimSpazioBianco,altezza_piano_0])(R([1,2])(PI)(getScalinataPosteriore(altezza_primo_piano+spessore_muri+0.1,4,25,8.75,4,3,0,3)))
    Scalinata2 = T([1,2,3])([-dimensione_pannello_finestra*2.0,-2+(dimensione_lato_frontale_edificio+DimSpazioBianco*2.0+spessore_muri)-(dimensione_pannello_finestra*2+DimSpazioBianco),altezza_piano_0])(R([1,2])(PI)(getScalinataPosteriore(altezza_primo_piano+spessore_muri+0.1,4,25,8.75,4,3,3,0)))
    
    # scalinate tra piano 0 e primo piano
    Scalinata01 = T([1,2,3])([-dimensione_pannello_finestra+4,2+dimensione_pannello_finestra*2.0+DimSpazioBianco-4,-spessore_muri])(getScalinataPosteriore(altezza_piano_0+spessore_muri+0.1,4,15,4,4,3,0,0))
    Scalinata02 = T([1,2,3])([-dimensione_pannello_finestra+4,-2+(dimensione_lato_frontale_edificio+DimSpazioBianco*2.0+spessore_muri+4)-(dimensione_pannello_finestra*2+DimSpazioBianco),-spessore_muri])(getScalinataPosteriore(altezza_piano_0+spessore_muri+0.1,4,15,4,4,3,0,0))
    
    Scale = STRUCT([Scalinata1,Scalinata2,Scalinata01,Scalinata02])
    
    PareteP2_4 = STRUCT([PareteP2_4,T(1)(-dimensione_pannello_finestra*2.0)(PareteP2_5)])
    
    # costruzione interni della Villa
    Interni = T(3)(-0.1)(STRUCT([PareteP2_4,DueParete2,DuePareteD,Porta,Scale,MuroVicinoScale]))

    return TEXTURE(texture_pareti)(Interni)

In [152]:
VIEW(getParetiInterne(dimensione_laterale_edificio,dimensione_lato_frontale_edificio,altezza_pareti))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D8E5750> >

<table>
    <tr><td>![interni.png](imm/interni.png)</td>
    <td>![interni.png](imm/interni-2.png)</td></tr>
</table>

In [61]:
VIEW(STRUCT([TOP([Prato,Piano0]),
             T([1,2])([posizione_piano2_X+dimensione_laterale_edificio-dimensione_pannello_finestra,posizione_piano2_Y])(getParetiInterne(dimensione_laterale_edificio,dimensione_lato_frontale_edificio,altezza_pareti))]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D1F7690> >

<table>
    <tr><td>![interni2.png](imm/interni2.png)</td>
    <td>![interni2.png](imm/interni2-2.png)</td></tr>
</table>

In [62]:
VIEW(STRUCT([TOP([TOP([Prato,Piano0]),Piano1]),
             T([1,2,3])([posizione_piano2_X,posizione_piano2_Y,altezza_secondo_piano+0.1])(Piano2),
             T([1,2,3])([posizione_tetto_X,posizione_tetto_Y,altezza_tetto+0.1])(Tetto),
             T([1,2,3])([posizione_facciata_Y+spessore_facciata,dim_prato_y/2.0,altezza_piano_0-altezza_scalinata_frontale+0.1])(Scalinata),
             T([1,2])([posizione_piano2_X,dim_prato_y/2.0])(Scalinata_posteriore),
             T([1,2,3])([posizione_facciata_Y,posizione_facciata_X,posizione_facciata_Z])(Facciata),
             T([1,2])([posizione_piano2_X+dimensione_laterale_edificio-dimensione_pannello_finestra,posizione_piano2_Y])(getParetiInterne(dimensione_laterale_edificio,dimensione_lato_frontale_edificio,altezza_pareti))]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D15FC60> >

<table>
    <tr><td>![interni3.png](imm/interni3.png)</td>
    <td>![interni3.png](imm/interni3-2.png)</td></tr>
    <tr><td>![interni3.png](imm/interni3-3.png)</td>
    <td>![interni3.png](imm/interni3-4.png)</td></tr>
    <tr><td>![interni3.png](imm/interni3-5.png)</td>
    <td>![interni3.png](imm/interni3-6.png)</td></tr>
</table>

### Definizione dell'albero
- getAlbero(altezza,larghezza,x,y,colore) restituisce un albero come quello presente in figura in posizione (x,y) di altezza, larghezza e colore specificata;
- getForesta(numero_alberi,altezza_minima,altezza_massima,larghezza_minima,larghezza_massima) restituisce un insieme di alberi di diversa dimensione a seconda dei parametri inseriti. Per fare qeusto sono state utilizzate diverse funzioni di Numpy;
- spazioVerde(x1,x2,x3,y1,y2,y3) restituisce un punto sul piano (x,y) randomico, ma sicuramente occupato dal prato. Lo spazio libero viene definito in questo modo: si parte da x1 e si arriva fino a x2, poi viene saltato una spazio x3 e si arriva fino alla fine. Lo stesso viene fatto sull'asse y. Il risultato è che lo spazio libero è quello dell'intero prato a qui viene sottratto lo spazio occupato dalla Villa;
- anche in questo caso x1,x2,x3,y1,y2 e y3 sono passatti come parametri.

In [63]:
def getAlbero(altezza,larghezza,x,y,colore):
    
    if (colore==0):
        texture_foglie = texture_foglie1
    elif (colore==1):
        texture_foglie = texture_foglie2
    else:
        texture_foglie = texture_foglie3
    
    V1 = [[0,0],[larghezza,0],[larghezza/2.0,altezza*0.6],[larghezza/6.0,altezza*0.35*0.5],[larghezza/3.0,altezza*0.35*0.5],[larghezza*2.0/3.0,altezza*0.35*0.5],[larghezza*5.0/6.0,altezza*0.35*0.5]]
    
    FV1 = [[3,6,2],[4,5,0],[0,4,5,1]]
    
    Foglie2 = T(1)(-larghezza/2.0)(STRUCT(MKPOLS((V1,FV1))))
    Foglie1 = TEXTURE(texture_foglie)(STRUCT([R([1,3])(PI)(Foglie2),T(3)(0.05)(Foglie2)]))
    
    Foglie = T(3)(altezza*0.30)(R([2,3])(PI/2)(COMP([STRUCT, NN(4)])([Foglie1,R([1,3])(PI/4)])))
    
    Tronco = TEXTURE(texture_tronco)(CONE([larghezza/12.0, altezza*0.60])(24))
    
    Albero = T([1,2])([x,y])(STRUCT([Foglie,Tronco]))
    
    return (Albero)

#restituisce delle cordinate di prato libero
def spazioVerde(x1,x2,x3,y1,y2,y3):
    x = np.random.randint(x1,x2)+x3*np.random.randint(0,2)
    y = np.random.randint(y1,y2)+y3*np.random.randint(0,2)
    return x,y

def getForesta(numero_alberi,altezza_minima,altezza_massima,larghezza_minima,larghezza_massima):
    coordinate = [spazioVerde(larghezza_massima,dim_prato_x,0,larghezza_massima,posizione_piano2_Y-larghezza_massima,posizione_piano2_Y+dimensione_lato_frontale_edificio+larghezza_massima) for i in np.arange(numero_alberi)]
    alberi = [getAlbero(np.random.randint(altezza_minima,altezza_massima+1),np.random.randint(larghezza_minima,larghezza_massima+1),coordinate[i][0],coordinate[i][1],np.random.randint(0,3)) for i in np.arange(numero_alberi)]
    
    return STRUCT(alberi)

In [64]:
VIEW(getAlbero(10,6,0,0,0))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D2A0450> >

<tr><td>![albero.png](imm/albero.png)</td>
    <td>![albero.png](imm/albero-2.png)</td></tr>

In [65]:
VIEW(TOP([Prato,getForesta(20,8,10,5,7)]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D2A05A0> >

<tr><td>![foresta.png](imm/foresta.png)</td>
    <td>![foresta.png](imm/foresta-2.png)</td></tr>

# Risultato conclusivo

In [66]:
VIEW(STRUCT([TOP([STRUCT([PratoConCollina,T([1,2,3])([posizione_piano2_X,posizione_piano2_Y,0.1])(Piano0)]),Piano1]),
             T([1,2,3])([posizione_piano2_X,posizione_piano2_Y,altezza_secondo_piano+0.1])(Piano2),
             T([1,2,3])([posizione_tetto_X,posizione_tetto_Y,altezza_tetto+0.1])(Tetto),
             T([1,2,3])([posizione_facciata_Y+spessore_facciata,dim_prato_y/2.0,altezza_piano_0-altezza_scalinata_frontale+0.1])(Scalinata),
             T([1,2])([posizione_piano2_X,dim_prato_y/2.0])(Scalinata_posteriore),
             T([1,2,3])([posizione_facciata_Y,posizione_facciata_X,posizione_facciata_Z])(Facciata),
             T([1,2])([posizione_piano2_X+dimensione_laterale_edificio-dimensione_pannello_finestra,posizione_piano2_Y])(getParetiInterne(dimensione_laterale_edificio,dimensione_lato_frontale_edificio,altezza_pareti)),
             (getForesta(50,8,13,4,6))
            ]))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000D2A0D50> >

<table>
    <tr><td>![villa.png](imm/villa-a.png)</td></tr>
    <tr><td>![villa.png](imm/villa-2-a.png)</td></tr>
    <tr><td>![villa.png](imm/villa-3.png)</td></tr>
    <tr><td>![villa.png](imm/villa-4.png)</td></tr>
</table>